In [10]:
import urllib
from datetime import datetime
import pandas as pd
import os

In [11]:
D = ['Вінницька', 'Волинська', 'Дніпропетровська', 'Донецька', 'Житомирська',
     'Закарпатська', 'Запорізька', 'Івано-Франківська', 'Київська', 'Кіровоградська',
     'Луганська', 'Львівська', 'Миколаївська', 'Одеська', 'Полтавська',
     'Рівненська', 'Сумська', 'Тернопільска', 'Харківська', 'Херсонська',
     'Хмельницька', 'Черкаська', 'Чернівецька', 'Чернігівська', 'Республіка Крим']

In [12]:
def VHIDownload():
   LST=[24, 25, 5, 6, 27, 23, 26, 7, 11, 13, 14, 15, 16, 17, 18, 19, 21, 22, 8, 9, 10, 1, 3, 2, 20]
   for number, obl in enumerate(LST, 1):
        url=f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={obl}&year1=1981&year2=2020&type=Mean'

        wp = urllib.request.urlopen(url)
        text = wp.read()

        date = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
        out = open(f'VHI_ID_{number} {date}.csv', 'wb')
        out.write(text)
        out.close()

In [13]:
def CreateDataFrame(path):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    all_files = [i for i in os.listdir(path) if i.endswith('csv')]
    dataFrames = []
    for filename in all_files:
        df = pd.read_csv(fr'{path}\{filename}', header = 1, names = headers)
        df = df.drop(df.loc[df['VHI'] == -1].index)
        i = int(filename.split('_')[2].split()[0])
        df['area'] = D[i-1]

        dataFrames.append(df)    
    dataFrame = pd.concat(dataFrames, ignore_index=True)
    
    dataFrame.pop('empty')
    dataFrame = dataFrame.dropna()
    dataFrame['Year'][(dataFrame['Year']=='<tt><pre>1982')] = '1982'
    dataFrame['Week'] = dataFrame['Week'].astype(int)
    return dataFrame

In [14]:
def ExtremumSearch(df, area, year):
    Min = df[(df['Year'] == str(year)) & (df['area'] == area)].VHI.min()
    Max = df[(df['Year'] == str(year)) & (df['area'] == area)].VHI.max()
    print(f'The minimum value of VHI for {area} область in {year} = {Min}')
    print(f'The maximum value of VHI for {area} область in {year} = {Max}')


In [15]:
def ExtremeDroughts(df, area):
    df_droughts = sorted(set(df[(df['area'] == area) & (df['VHI'] <= 15)]['Year']))
    if len(df_droughts) == 0:
        print(f'There were no extreme droughts in the region {area}')
    else:
        print(f'Extreme droughts for {area} область were in', end = ' ')
        print(*df_droughts, sep=', ')

In [16]:
def ModerateDrought(df, area):
    df_droughts = sorted(set(df[(df['area'] == area) & (df['VHI'] <= 35) & (15 < df['VHI'])]['Year']))
    if len(df_droughts) == 0:
        print(f'There were no moderate droughts in the region {area}')
    else:
        print(f'Moderate droughts for {area} область were in', end = ' ')
        print(*df_droughts, sep=', ')

In [17]:
path = r'C:\Users\Костянтин\uni\Lab1'
df = CreateDataFrame(path)
df.head()

,Year,Week,SMN,SMT,VCI,TCI,VHI,area
0,1982,1,0.068,263.59,63.47,28.34,45.90,Вінницька
1,1982,2,0.074,265.78,67.62,23.05,45.34,Вінницька
2,1982,3,0.076,267.19,69.37,20.40,44.88,Вінницька
3,1982,4,0.075,268.57,65.26,17.93,41.60,Вінницька
4,1982,5,0.072,269.24,58.58,20.00,39.29,Вінницька


In [18]:
ExtremumSearch(df, 'Київська', 2007)

The minimum value of VHI for Київська область in 2007 = 40.1
The maximum value of VHI for Київська область in 2007 = 52.7


In [19]:
ExtremeDroughts(df, 'Одеська')

Extreme droughts for Одеська область were in 2007


In [20]:
ModerateDrought(df, 'Кіровоградська')

Moderate droughts for Кіровоградська область were in 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2003, 2006, 2007, 2008, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020
